In [ ]:
from imutils.video import VideoStream
from imutils import face_utils
import datetime
import argparse
import imutils
import time
import dlib
import cv2
import glob
import sys
import os
import shutil 
import math
import numpy as np
from albumentations.augmentations import transforms
import skimage
from tqdm import tqdm

from skimage.transform import resize
from sklearn.model_selection import train_test_split
from scipy import stats as s

import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.xception import Xception, preprocess_input
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.layers import Dense, InputLayer, Dropout, Flatten, Activation, GRU
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, Convolution2D, ZeroPadding2D, Bidirectional, TimeDistributed
from keras.layers.recurrent import LSTM
from keras.utils import np_utils
from keras.preprocessing import image, sequence
from keras.callbacks import ModelCheckpoint


#from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import optimizers

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
base_dir = 'data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')
#os.makedirs(base_dir)
#os.makedirs(train_dir)
#os.makedirs(val_dir)
#os.makedirs(test_dir)

In [ ]:
classes = 'Begin, Choose, Connection, Navigation, Next, Previous, Start, Stop, Hello, Web'
classes = classes.split(', ')
classes = classes[:6]
classes

In [ ]:
for class_name in classes:
    train_vids_dir = os.path.join(train_dir, class_name)
    val_vids_dir = os.path.join(val_dir, class_name)
    test_vids_dir = os.path.join(test_dir, class_name)
    
    os.makedirs(train_vids_dir)
    os.makedirs(val_vids_dir)
    os.makedirs(test_vids_dir)

In [ ]:
predictor_path = 'shape_predictor_68_face_landmarks.dat'

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

In [ ]:
train_people = 'F01, F02, F04, F05, F06, F07, F08, F09, M01, M04'.split(', ')
#train_people = 'F01, F02, F05, F06, F07, F09, M01, M04'.split(', ')
val_people = 'F10, M07'.split(', ')
test_people = 'F11, M08'.split(', ')
print(train_people)
print(val_people)
print(test_people)

classes_num = ['0'+str(i) if i < 10 else str(i) for i in range(1, 11) ]
classes_num = classes_num[:6]
print(classes_num)
word_ids = ['0'+str(i) if i < 10 else str(i) for i in range(1, 11) ]
print(word_ids)

In [ ]:
classes_dict = dict(zip(classes_num, classes))
classes_dict['01']

In [ ]:
def show_img(img, figsize=(8, 8)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.grid(False)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.imshow(img)
    plt.imshow(img)

In [ ]:
counter = 0
for classi in classes_num[0]:
    for person in tqdm(train_people):
        for word_id in word_ids:
            for f in sorted(glob.glob(os.path.join('data/miracl/'+person+'/words/'+classi+'/'+ word_id, "*.jpg"))):
                img = cv2.imread(f, 1)
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                rects = detector(gray, 0)
                
                for k, rect in enumerate(rects):
                    x1 = rect.left()
                    y1 = rect.top()
                    x2 = rect.right()
                    y2 = rect.bottom()
                    
                    offset_x = (100-(abs(x1-x2)))/2
                    offset_y = (100-(abs(y1-y2)))/2
                    
                    img = img[int(y1-offset_y):int(y2+offset_y), int(x1-offset_x):int(x2+offset_x)]
                
                counter += 1
                
                #cv2.imwrite('data/train/' + classes_dict[classi] + '/' + classi + '_' 
                #            + person + '_' 
                #            + word_id + '_' 
                #            + 'frame' + f[29:-4] + '.jpg', img)
                cv2.imwrite('data/train/' 
                            + classes_dict[classi] + '/' 
                            + classi  + '_' 
                            + person  + '_' 
                            + word_id + '_' 
                            + f[28:-4] + '.jpg', img)

In [ ]:
def get_idx(classi):
    start_idx = (len('data/train/') + len(classi)+1)
    return start_idx

In [ ]:
print(get_idx('Web'))

In [ ]:
for classi in classes:
    files = [file for file in sorted(glob.glob(os.path.join('data/train/' + classi + '/*.jpg')))]
    files_clean = [file_clean for file_clean in sorted(files)
                   if 'vert_flip' not in file_clean
                   and 'hor_flip' not in file_clean
                   and 'rot_45' not in file_clean
                   and 'rot_315' not in file_clean
                   and 'rot_135' not in file_clean
                   and 'rot_225' not in file_clean
                   and 'rot_90' not in file_clean
                   and 'rot_270' not in file_clean
                   and 'rand_contr' not in file_clean
                   and 'noised' not in file_clean]
    files_rem = [file_rem for file_rem in sorted(files)
                   if 'vert_flip' in file_rem
                   or 'hor_flip' in file_rem
                   or 'rot_45' in file_rem
                   or 'rot_315' in file_rem
                   or 'rot_135' in file_rem
                   or 'rot_225' in file_rem
                   or 'rot_90' in file_rem
                   or 'rot_270' in file_rem
                   or 'rand_contr' in file_rem
                   or 'noised' in file_rem]
    for file_to_rem in files_rem: os.remove(file_to_rem)
    files = [file for file in sorted(glob.glob(os.path.join('data/train/' + classi + '/*.jpg')))]
    print(classi, 'dir: removed', len(files_rem), 'files.', len(files), 'to be augmented')

    for file_to_hor_flip in files_clean:
        img = cv2.imread(file_to_hor_flip, 1)
        img = transforms.HorizontalFlip().apply(img)
        file_to_write = file_to_hor_flip[:get_idx(classi)] + '[hor_flip]' + file_to_hor_flip[get_idx(classi):]
        cv2.imwrite(file_to_write, img)
    files = [file for file in sorted(glob.glob(os.path.join('data/train/' + classi + '/*.jpg')))]
    files_hor_flipped = [file_hor_flipped for file_hor_flipped in sorted(files) if 'hor_flip' in file_hor_flipped]
    print('Horizontally flipped', len(files_hor_flipped), 'files.', len(files), 'in the directory')
    
    for file_to_vert_flip in files_clean:
        img = cv2.imread(file_to_vert_flip, 1)
        img = transforms.VerticalFlip().apply(img)
        file_to_write = file_to_vert_flip[:get_idx(classi)] + '[vert_flip]' + file_to_vert_flip[get_idx(classi):]
        cv2.imwrite(file_to_write, img)
    files = [file for file in sorted(glob.glob(os.path.join('data/train/' + classi + '/*.jpg')))]
    files_vert_flipped = [file_vert_flipped for file_vert_flipped in sorted(files) if 'vert_flip' in file_vert_flipped]
    print('Vertically flipped', len(files_vert_flipped), 'files.', len(files), 'in the directory')
    
    #files_not_noised = [f for f in sorted(glob.glob(os.path.join('data/train/' + classi + '/*.jpg'))) if 'noised' not in f ]    
    #for file_to_rand_contr in files_not_noised:
    #    img = cv2.imread(file_to_rand_contr, 1)
    #    img = transforms.RandomContrast().apply(img)
    #    file_to_write = file_to_rand_contr[:get_idx(classi)] + '[rand_contr]' + file_to_rand_contr[get_idx(classi):]
    #    #print(file_to_rand_contr)
    #    #print(file_to_write)
    #    cv2.imwrite(file_to_write, img)
    #files = [file for file in sorted(glob.glob(os.path.join('data/train/' + classi + '/*.jpg')))]
    #files_rand_contred = [file_rand_contred for file_rand_contred in sorted(files) if 'rand_contr' in file_rand_contred]
    #print('Random contrasted', len(files_rand_contred), 'files.', len(files), 'in the directory')
    
    files_not_rand_contred = [file_not_rand_contred for file_not_rand_contred in sorted(files) if 'rand_contr' not in file_not_rand_contred]
    for file_to_noise in files_not_rand_contred:
        if '[hor_flip]' in file_to_noise:
            img = cv2.imread(file_to_noise, 1)
            gauss = np.random.uniform(0, 64, img.size)
            gauss = gauss.reshape(img.shape[0],img.shape[1],img.shape[2]).astype('uint8')
            img = cv2.add(img,gauss)
            file_to_write = file_to_noise[:get_idx(classi)] + '[noised]' + file_to_noise[get_idx(classi):]
            #print(file_to_noise)
            #print(file_to_write)
            cv2.imwrite(file_to_write, img)
        elif '[vert_flip]' in file_to_noise:
            img = cv2.imread(file_to_noise, 1)
            gauss = np.random.uniform(0, 64, img.size)
            gauss = gauss.reshape(img.shape[0],img.shape[1],img.shape[2]).astype('uint8')
            img = cv2.add(img,gauss)
            file_to_write = file_to_noise[:get_idx(classi)] + '[noised]' + file_to_noise[get_idx(classi):]
            cv2.imwrite(file_to_write, img)
        else:
            img = cv2.imread(file_to_noise, 1)
            gauss = np.random.uniform(0, 64, img.size)
            gauss = gauss.reshape(img.shape[0],img.shape[1],img.shape[2]).astype('uint8')
            img = cv2.add(img,gauss)
            file_to_write = file_to_noise[:get_idx(classi)] + '[noised]' + file_to_noise[get_idx(classi):]
            cv2.imwrite(file_to_write, img)
    files = [file for file in sorted(glob.glob(os.path.join('data/train/' + classi + '/*.jpg')))]
    files_noised = [file_noised for file_noised in sorted(files) if 'noised' in file_noised]
    print('Noised', len(files_noised), 'files.', len(files), 'in the directory')
    
    print()

In [ ]:
counter = 0
for classi in classes_num:
    for person in tqdm(val_people):
        for word_id in word_ids:
            for f in sorted(glob.glob(os.path.join('data/miracl/'+person+'/words/'+classi+'/'+ word_id, "*.jpg"))):
            #for f in sorted(glob.glob(os.path.join('MIRACL/'+person+'/words/'+classi+'/'+ word_id, "*.jpg"))):
                img = cv2.imread(f, 1)
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                rects = detector(gray, 0)
                
                for k, rect in enumerate(rects):
                    x1 = rect.left()
                    y1 = rect.top()
                    x2 = rect.right()
                    y2 = rect.bottom()
                    
                    offset_x = (100-(abs(x1-x2)))/2
                    offset_y = (100-(abs(y1-y2)))/2
                    
                    img = img[int(y1-offset_y):int(y2+offset_y), int(x1-offset_x):int(x2+offset_x)]
                
                counter += 1
                
                #cv2.imwrite('data/validation/' 
                #            + classes_dict[classi] + '/' 
                #            + classi + '_' 
                #            + person + '_' 
                #            + word_id + '_' 
                #            + 'frame' + f[29:-4] + '.jpg', img)
                
                cv2.imwrite('data/validation/' 
                            + classes_dict[classi] + '/' 
                            + classi  + '_' 
                            + person  + '_' 
                            + word_id + '_' 
                            + f[28:-4] + '.jpg', img)

In [ ]:
train_image = []
train_class = []

for class_id in tqdm(range(len(classes))):
    images = sorted(glob.glob('data/train/' + classes[class_id] + '/*.jpg'))
    for i in range(len(images)):
        if ('noised' in images[i] or 'rand_contr' in images[i]
        or 'vert_flip' in images[i] or 'hor_flip' in images[i]):
            train_image.append(images[i].split('/')[3])
            #print(images[i].split('/')[3])
            train_class.append(classes_dict[images[i].split('/')[3].split(']')[-1][:2]])
            #print(images[i].split('/')[3].split(']')[-1][:2])
        else:
            #print(images[i])
            train_image.append(images[i].split('/')[3])
            #print(images[i].split('/')[3])
            train_class.append(classes_dict[images[i].split('/')[3].split('_')[0]])
            #print(images[i].split('/')[3].split('_')[0])
        
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

train_data.to_csv('data/miracl/train_new.csv',header=True, index=False)

In [ ]:
train = pd.read_csv('data/miracl/train_new.csv')
train.head()

In [ ]:
train.tail()

In [ ]:
len(train)

In [ ]:
y_tr = train['class']
y_tr_dummy = pd.get_dummies(y_tr)

In [ ]:
val_image = []
val_class = []

for class_id in tqdm(range(len(classes))):
    images = sorted(glob.glob('data/validation/' + classes[class_id] + '/*.jpg'))
    for i in range(len(images)):
        val_image.append(images[i].split('/')[3])
        val_class.append(classes_dict[images[i].split('/')[3].split('_')[0]])
val_data = pd.DataFrame()
val_data['image'] = val_image
val_data['class'] = val_class

val_data.to_csv('data/miracl/val_new.csv',header=True, index=False)

In [ ]:
val = pd.read_csv('data/miracl/val_new.csv')
val.head()

In [ ]:
val.tail()

In [ ]:
len(val)

In [ ]:
y_val = val['class']
y_val_dummy = pd.get_dummies(y_val)

In [ ]:
print(len(train))
print(len(val))

In [ ]:
conv_base = MobileNet(weights='imagenet', include_top=False, input_shape=(100,100,3), pooling='avg')
#conv_base = MobileNet(weights='imagenet', include_top=False, input_shape=(100,100,3))

In [ ]:
conv_base.summary()

In [ ]:
import sys

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 64

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 1024)) #like in global_average_pooling2d_1
    labels = np.zeros(shape=(sample_count, 6)) # 6 - num of classes
    generator = datagen.flow_from_directory(
        directory,
        target_size=(100, 100),
        batch_size=batch_size,
        class_mode='categorical')
    
    i = 0
    
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        sys.stdout.write('\rProcessed: %d, size of features: %f' %((i * batch_size), sys.getsizeof(features)*1e-6))
        sys.stdout.flush()
        if i * batch_size >= sample_count:
            break
    
    return features, labels

In [ ]:
#TODO tqdm(range(len(....))

train_features, train_labels = extract_features(train_dir, len(train))

In [ ]:
validation_features, validation_labels = extract_features(val_dir, len(val))

In [ ]:
print(train_features.shape)
print(validation_features.shape)

In [ ]:
import sys
print(sys.getsizeof(train_features)*1e-6)
print(sys.getsizeof(validation_features)*1e-6)

In [ ]:
# flatten them to (samples, 16384):
# train_features = np.reshape(train_features, (41742, 3 * 3 * 1024))
# validation_features = np.reshape(validation_features, (1746, 3 * 3 * 1024))

In [ ]:
print(train_features.shape)
print(validation_features.shape)

In [ ]:
train_features_rnn = np.reshape(train_features, (train_features.shape[0], 1, train_features.shape[1]))


validation_features_rnn = np.reshape(validation_features, (validation_features.shape[0], 1, validation_features.shape[1]))

In [ ]:
print(train_features_rnn.shape)
print(validation_features_rnn.shape)

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(1, 1024)))
model.add(Dense(6, activation='softmax'))

In [ ]:
#model = Sequential()
#model.add(LSTM(256, input_shape=(1, 9216)))
#model.add(Dense(10, activation='softmax'))

In [ ]:
model_fc = Sequential()
#model_fc.add(Dense(128, activation='relu', input_dim=3 * 3 * 1024))
model_fc.add(Dense(128, activation='relu', input_dim=1024))
model_fc.add(Dropout(0.3))
model_fc.add(Dense(6, activation='softmax'))

In [ ]:
from keras.callbacks import ModelCheckpoint
adam2 = optimizers.Adam(lr=0.001)
save_weights = ModelCheckpoint('weight_fc.hdf5', save_best_only=True, monitor='val_loss', mode='min')
model_fc.compile(loss='categorical_crossentropy',optimizer=adam2,metrics=['accuracy'])
model_fc.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint
adam2 = optimizers.Adam(lr=0.001)
save_weights = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')
model.compile(loss='categorical_crossentropy',optimizer=adam2,metrics=['accuracy'])
model.summary()

In [ ]:
history_fc = model_fc.fit(train_features, train_labels,
                    epochs=20,
                    batch_size=256,
                    callbacks=[save_weights],
                    validation_data=(validation_features, validation_labels))

In [ ]:
history = model.fit(train_features_rnn, train_labels,
                    epochs=20,
                    batch_size=256,
                    callbacks=[save_weights],
                    validation_data=(validation_features_rnn, validation_labels))

In [ ]:
model.load_weights("weight.hdf5")

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=adam2,metrics=['accuracy'])

In [ ]:
counter = 0
for classi in classes_num:
    for person in tqdm(test_people):
        for word_id in word_ids:
            #for f in sorted(glob.glob(os.path.join('MIRACL/'+person+'/words/'+classi+'/'+ word_id, "*.jpg"))):
            for f in sorted(glob.glob(os.path.join('data/miracl/'+person+'/words/'+classi+'/'+ word_id, "*.jpg"))):
                img = cv2.imread(f, 1)
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                rects = detector(gray, 0)
                
                for k, rect in enumerate(rects):
                    x1 = rect.left()
                    y1 = rect.top()
                    x2 = rect.right()
                    y2 = rect.bottom()
                    
                    offset_x = (128-(abs(x1-x2)))/2
                    offset_y = (128-(abs(y1-y2)))/2
                    
                    img = img[int(y1-offset_y):int(y2+offset_y), int(x1-offset_x):int(x2+offset_x)]
                
                counter += 1
                
                cv2.imwrite('data/test_miracl/' + classi + '_' 
                            + person + '_' 
                            + word_id + '_' 
                            + 'frame' + f[29:-4] + '.jpg', img)

In [ ]:
images = sorted(glob.glob("data/test_miracl/*.jpg"))
test_image = []
test_class = []
for i in tqdm(range(len(images))):
    #print(images[i])
    test_image.append(sorted(images)[i].split('/')[2])
    test_class.append(sorted(images)[i].split('/')[2][:2])


test_data = pd.DataFrame()
test_data['image'] = test_image
test_data['class'] = test_class
test_images = test_data['image']

test_data.to_csv('test_new.csv',header=True, index=False)

In [ ]:
test = pd.read_csv('test_new.csv')
y_test = test['class']
y_test = pd.get_dummies(y_test)

In [ ]:
test.head()

In [ ]:
test.tail()

In [ ]:
video_names = sorted(list(set([test_images[i].split('.')[0][0:9] for i in range(len(test_images))])))
test_videos = pd.DataFrame(video_names)

test_videos.columns = ['dummy']
test_videos = test_videos['dummy']

print(test_videos.shape[0])
test_videos.head()

In [ ]:
print(test_images.shape[0])
test_images.tail()

In [ ]:
predict = []
actual = []

for video_id in tqdm(range(test_videos.shape[0])):
    videoFile = test_videos[video_id]
    
    files = glob.glob('temp/*')
    for f in files:
        os.remove(f)
    
    for frame_id in range(test_images.shape[0]):
        frame = test_images[frame_id]
        if test_images[frame_id].split('.')[0][0:9] == videoFile:
            img = shutil.copyfile('data/test_miracl/'+test_images[frame_id], 'temp/'+test_images[frame_id]) 
    
    images = glob.glob("temp/*.jpg")
    
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(100,100,3))
        img = image.img_to_array(img)
        img = keras.applications.mobilenet.preprocess_input(img)
        
        prediction_images.append(img)
        
    prediction_images = np.array(prediction_images)
    
    prediction_images = conv_base.predict(prediction_images)
    
    #prediction_images = prediction_images.reshape(prediction_images.shape[0], 1024)
    
    #prediction_images = np.reshape(prediction_images, 
    #                               (prediction_images.shape[0], 1, prediction_images.shape[1])) #(x, 1, 1024)
    
    prediction = model_fc.predict_classes(prediction_images)
    predict.append(y_test.columns.values[s.mode(prediction)[0][0]])
    
    actual.append(int(videoFile.split('_')[0].lstrip('0')))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100